<a href="https://colab.research.google.com/github/nguyenhoanganh2002/Optimal-planning/blob/main/CBUS_ortools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ortools


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Mixed Interger Programing**

In [ ]:
from ortools.linear_solver import pywraplp
import numpy as np
from itertools import combinations

In [ ]:
solver = pywraplp.Solver.CreateSolver("SCIP")

n, k = 8, 3

c = [[0, 7, 5, 6, 1, 7, 6, 2, 9, 5, 10, 10, 10, 1, 8, 5, 4],
 [7, 0, 9, 4, 10, 7, 7, 7, 7, 5, 10, 1, 3, 8, 6, 9, 1],
 [5, 9, 0, 9, 3, 8, 1, 7, 10, 10, 2, 4, 8, 3, 10, 5, 5],
 [6, 4, 9, 0, 10, 2, 7, 1, 10, 8, 7, 2, 4, 1, 2, 5, 6],
 [1, 10, 3, 10, 0, 5, 8, 8, 8, 4, 8, 6, 8, 3, 2, 4, 2],
 [7, 7, 8, 2, 5, 0, 7, 8, 1, 1, 6, 1, 3, 9, 1, 6, 7],
 [6, 7, 1, 7, 8, 7, 0, 4, 4, 8, 3, 5, 6, 8, 9, 4, 5],
 [2, 7, 7, 1, 8, 8, 4, 0, 1, 3, 6, 10, 6, 3, 8, 5, 6],
 [9, 7, 10, 10, 8, 1, 4, 1, 0, 6, 8, 9, 1, 3, 3, 6, 6],
 [5, 5, 10, 8, 4, 1, 8, 3, 6, 0, 7, 6, 2, 5, 7, 5, 10],
 [10, 10, 2, 7, 8, 6, 3, 6, 8, 7, 0, 8, 1, 10, 6, 5, 6],
 [10, 1, 4, 2, 6, 1, 5, 10, 9, 6, 8, 0, 5, 4, 4, 4, 7],
 [10, 3, 8, 4, 8, 3, 6, 6, 1, 2, 1, 5, 0, 8, 1, 5, 6],
 [1, 8, 3, 1, 3, 9, 8, 3, 3, 5, 10, 4, 8, 0, 2, 9, 10],
 [8, 6, 10, 2, 2, 1, 9, 8, 3, 7, 6, 4, 1, 2, 0, 2, 5],
 [5, 9, 5, 5, 4, 6, 4, 5, 6, 5, 5, 4, 5, 9, 2, 0, 3],
 [4, 1, 5, 6, 2, 7, 5, 6, 6, 10, 6, 7, 6, 10, 5, 3, 0]]

###### variables -----------------------------------------------------------
x = []
for i in range(2*n + 1):
    x_i = []
    for j in range(2*n + 1):
        if i == j:
            x_i.append(0)
        else:
            x_i.append(solver.IntVar(0, 1, f'x[{i}][{j}]'))
    x.append(x_i)

y = [solver.IntVar(1, 2*n + 1, f'y[{i}]') for i in range(2*n + 1)]

p = [solver.IntVar(0, k, f'p[{i}]') for i in range(2*n + 1)]

###### constrains ----------------------------------------------------------
# one way in, one way out
for i in range(2*n + 1):
    solver.Add(sum(x[i][j] for j in range(2*n + 1)) == 1)
    solver.Add(sum(x[j][i] for j in range(2*n + 1)) == 1)

# not subcycles
for i in range(2*n + 1):
    for j in range(1, 2*n + 1):
        if i == j:
            continue
        solver.Add(y[i] - y[j] + (2*n + 1)*x[i][j] <= 2*n)

# start at city 0
solver.Add(y[0] == 1)

# picks before drops
for i in range(n+1, 2*n + 1):
    solver.Add(y[i] >= (y[i - n] + 1))

# n of passenger at city 0 equals 0
solver.Add(p[0] == 0)

# maximum k passengers
for i in range(2*n + 1):
    for j in range(1, 2*n+1):
        if i == j:
            continue
        # picks
        if j <= n:
            solver.Add(p[j] - p[i] >= 1 - (k + 1)*(1 - x[i][j]))
        # drops
        if j > n:
            solver.Add(p[i] - p[j] >= 1 - (k + 1)*(1 - x[i][j]))

# picks = [i for i in range(1, n+1)]
# for j in range(1, n - k + 1):
#     U_n = k + 1 + 2*j
#     U_1 = k + 2
#     threshold = U_n*(U_n + 1)/2 - 1 - j*(U_n - 1 + U_1)/2
#     combs = list(combinations(picks, k + j))
#     for comb in combs:
#         solver.Add(sum(y[idx] for idx in comb) >= threshold)



###### Objective -------------------------------------------------------
solver.Minimize(sum(sum(x[i][j]*c[i][j] for j in range(2*n + 1)) for i in range (2*n + 1)))

###### Solve -----------------------------------------------------------
stt = solver.Solve()
if stt == pywraplp.Solver.OPTIMAL:
    print("Solved")
    print(f"Objective value: {solver.Objective().Value()}")
    sol = {}
    for i in range(2*n + 1):
        sol[y[i].solution_value()] = i

    print(f"Solution: {[sol[i+1] for i in range(2*n + 1)]}")

Solved
Objective value: 29.0
Solution: [0, 4, 2, 6, 10, 12, 8, 7, 3, 11, 1, 16, 15, 14, 5, 9, 13]


In [ ]:
"""
Solved
Objective value: 29.0
Solution: [0, 4, 2, 6, 10, 12, 8, 7, 3, 14, 15, 16, 1, 11, 5, 9, 13]"""

# **Constrains Programming**

In [ ]:
"""
Solving CBUS problem using constraint programming
There are n passengers 1,2,...,𝑛 and 2n points
The passenger 𝑖 want to travel from point i to point 𝑖+𝑛(𝑖=1,2,...,𝑛).
There is a bus located at point 0 and has 𝑘 places for transporting the passengers (it means at any time, there are at most 𝑘passengers on the bus).
You are given the distance matrix c in which 𝑐(𝑖,𝑗) is the traveling distance from point 𝑖to point 𝑗(𝑖,𝑗=0,1,...,2𝑛).
Compute the shortest route for the bus, serving n passengers and coming back to point 0.
"""

from ortools.sat.python import cp_model
from random import randint

n = 6
K = 3

d = [[0 for i in range(0, 2*n+1)] for j in range(0, 2*n+1)]
for i in range(0, 2*n+1):
  for j in range(i+1, 2*n+1):
    d[i][j] = randint(1, 10)
    d[j][i] = d[i][j]


def main():
  model = cp_model.CpModel()
  pos_map = [[model.NewBoolVar('pos_map' + str(i) + '-' + str(j)) for j in range(2*n)] for i in range(2*n)]
  # each row will only have 1 True
  for i in range(2 * n):
    model.Add(sum(pos_map[i]) == 1)
  # each column will only have 1 True
  for j in range(2*n):
    model.Add(sum([pos_map[i][j] for i in range(2*n)]) == 1)

  # the pick up point of passenger i is before the drop off point of passenger i
  for j in range(n):
    model.Add(sum([pos_map[i][j] * i for i in range(2*n)]) <= sum([pos_map[i][j+n] * i for i in range(2*n)]))
  pass

  pick_or_drops = [model.NewIntVar(-1, 1, 'pick_or_drop' + str(i)) for i in range(2*n)]

  for i in range(2*n):
    b = model.NewBoolVar('b' + str(i))
    model.Add(pick_or_drops[i] == 1).OnlyEnforceIf(b)
    model.Add(pick_or_drops[i] == -1).OnlyEnforceIf(b.Not())
    model.Add(sum([pos_map[i][j] * j for j in range(2*n)]) < n).OnlyEnforceIf(b)
    model.Add(sum([pos_map[i][j] * j for j in range(2*n)]) >= n).OnlyEnforceIf(b)
  for i in range(2*n):
    model.Add(sum(pick_or_drops[:i]) <= K)

  coefs = [[[model.NewBoolVar('coef' + str(i) + '-' + str(j) + '-'+str(k)) for j in range(2*n)] for i in range(2*n)] for k in range(2*n)]
  for k in range(2*n-1):
    for i in range(n):
      for j in range(n):
        model.Add(pos_map[k][i] == 1).OnlyEnforceIf(coefs[k][i][j])
        model.Add(pos_map[k+1][j] == 1).OnlyEnforceIf(coefs[k][i][j])
  model.Minimize(
    sum([d[i][j] * coefs[k][i][j] for k in range(2*n-1) for i in range(2*n) for j in range(2*n)])
    + sum([d[0][i] * pos_map[0][i] for i in range(2*n)])
    + sum([d[i][0] * pos_map[2*n-1][i] for i in range(2*n)])
  )
  solver = cp_model.CpSolver()

  status = solver.Solve(model)
  if status == cp_model.OPTIMAL:
    print(solver.ObjectiveValue())
    print(str(n))
    for i in range(2*n):
      for j in range(2*n):
        if solver.Value(pos_map[i][j]):
          print(str(j+1) + " ", end="")
    print("")
  else:
    print('No solution found.')

main()


4.0
6
1 3 9 2 7 8 5 4 6 12 10 11 
